Inspiration: http://stackoverflow.com/questions/36378328/monte-carlo-simulation-in-python-parallelization-using-ipyparallel-taking-longe.

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy.random import random as random

In [2]:
class Packet():   
    
    def __init__(self, r, k, W, mua, mus, npackets):
        self.k = k
        self.r = r
        self.W = W
        self.mua = mua
        self.mus = mus
        self.npackets = npackets
        
        
    def absorb(self):
        mut = self.mua + self.mus
        dW = self.W * self.mua / mut
        
        self.W = self.W - dW
        
        return self.W

    def HG(self,g):   # HG = Henyey-Greenstein function for dust scattering phase funcion
        rand_theta = random(1)
        costheta = 0.5*(1+g**2-((1-g**2)/(1-g + 2.*g*rand_theta))**2)/g  # costheta gives the scattering angle theta
        return costheta

    def scatter(self):
        eps = 1.e-6
        
        # calculate new angle of scattering
        phi = 2*np.pi*random(1)        # phi is azimuthal angle, it can be anywhere between 0 and 2pi
        costheta = self.HG(0.85)   # it uses g = 0.85
        sintheta = (1-costheta**2)**0.5  

        sinphi = np.sin(phi) 
        cosphi = np.cos(phi)
        
        
        # Equations for new direction of propagation
        denom = (1-self.k[:,[2]]**2)**0.5
        
        # If the z-comp of direction is not close to 1:       
        mux = sintheta*(self.k[:,[0]]*self.k[:,[2]]*cosphi-self.k[:,[1]]*sinphi)/denom + self.k[:,[0]]*costheta 
        muy = sintheta*(self.k[:,[1]]*self.k[:,[2]]*cosphi+self.k[:,[0]]*sinphi)/denom + self.k[:,[1]]*costheta
        muz = -sintheta*cosphi*denom + self.k[:,[2]]*costheta
        
        # If the z-comp of direction is close to 1 and >= 0:
        ones = np.ones(mux.shape)        # I multiply by a matrix of ones to make it have the same dimensions as mux
        mux2 = sintheta*cosphi * ones
        muy2 = sintheta*sinphi * ones
        muz2 = costheta * ones
        
        # If the z-comp of direction is close to 1 and < 0:
        muy3 = -sintheta*sinphi * ones
        muz3 = -costheta * ones
            
        new_dir = np.where(denom > eps, (mux, muy, muz),
                  np.where(self.k[:,[2]]>= 0., (mux2, muy2, muz2), (mux2, muy3, muz3)))

        new_dir = np.transpose(new_dir[:,:,0])   

        
        # update the new direction of the packet 
        self.k[:,[0]] = new_dir[:,[0]]
        self.k[:,[1]] = new_dir[:,[1]]
        self.k[:,[2]] = new_dir[:,[2]]       

        return self.k  # returns updated direction of packet

    
    def move(self):
        # Determine step size L
        ksi = random(1)
        tau = -np.log(ksi)    # tau is optical depth
        L = tau / self.mus         # L is distance travelled by packet     
        
        self.r[:,[0]] = self.r[:,[0]] + L*self.k[:,[0]] 
        self.r[:,[1]] = self.r[:,[1]] + L*self.k[:,[1]] 
        self.r[:,[2]] = self.r[:,[2]] + L*self.k[:,[2]] 
                
        return self.r
    
    
    def bins(self, thickness):         
        eps = 1.e-6
        
        indices = []
        reflected_r = []
        transmitted_r = []
        reflected_k = []
        transmitted_k = []
        reflected_W = []
        transmitted_W = []
        
        # Check and delete the packets that exited the box or were completely absorbed
        
        for i in np.arange(self.npackets): 
            if self.r[i,2] > thickness:
                indices.append(i)
                transmitted_r.append(self.r[i,:])
                transmitted_k.append(self.k[i,:])
                transmitted_W.append(self.W[i,:])
                
            elif self.r[i,2] < 0.: 
                indices.append(i)
                reflected_r.append(self.r[i,:])
                reflected_k.append(self.k[i,:])
                reflected_W.append(self.W[i,:])
            
            elif self.W[i] < eps:
                indices.append(i)
                     
        for i in reversed(np.array(indices)):            
            self.r = np.delete(self.r, i, axis=0)
            self.k = np.delete(self.k, i, axis=0)
            self.W = np.delete(self.W, i, axis=0)
            self.npackets = len(self.r[:,[2]])
        
       
        return (np.array(reflected_r), np.array(reflected_k), np.array(reflected_W), 
        np.array(transmitted_r), np.array(transmitted_k), np.array(transmitted_W))
            

In [3]:
def initialize(npackets):

    ksi = random((npackets, 1))    
    costheta = np.sqrt(ksi)
    phi = 2*np.pi*ksi
    
    # Initial position
    r0 = np.zeros((npackets, 3))
    r0[:,[0]] = random((npackets,1))
    r0[:,[1]] = random((npackets,1))

    # Initial direction
    k0 = np.zeros((npackets, 3))
    k0[:,[0]] = np.sin(np.arccos(costheta)) * np.cos(phi)
    k0[:,[1]] = np.sin(np.arccos(costheta)) * np.sin(phi)
    k0[:,[2]] = costheta
    
    # Initial weight
    W0 = np.ones((npackets, 1)) * 0.001     # Figure out how to determine this
    
    # Absorption and scattering coefficients (figure out how to determine)
    mua = 0.0001
    mus = 0.01  
    
    return r0, k0, W0, mua, mus

In [4]:
# Run photon packets in parallel plane geometry

npackets = 2      # number of photon packets
thickness = 10.    # thickness of sample

r = initialize(npackets)[0]
k = initialize(npackets)[1]
W = initialize(npackets)[2]
mua = initialize(npackets)[3]
mus = initialize(npackets)[4]

packet = Packet(r, k, W, mua, mus, npackets)

while (packet.npackets > 0) :

    r = Packet.move(packet)
    
    W = Packet.absorb(packet)

    k = Packet.scatter(packet)
    
    bins = Packet.bins(packet, thickness)
    print(packet.npackets)

0


In [ ]:
bins